In [ ]:
import numpy as np

import pandas as pd
import lexis_functions as lf

# Read in Data

In [ ]:
df_full = pd.read_csv(r"C:/Users/jc4673/Documents/Columbia/CHS_Lexis/LexisNexis/LN_NOID_DATA/\
LN_Output_Voter_LN_InputLexisNexisCHSParticipantsNS.Dataset.csv",
                index_col=['ssn_altkey'])

In [ ]:
check = lambda x: 'special' in x or 'primary' in x or 'other' in x or 'general' in x or 'pres' in x

In [ ]:
# Segment to relevant categories
df = df_full[[x for x in df_full.columns.tolist() if check(x)]]
df.head()

In [ ]:
# Remove any completely empty columns
for col in df.columns.tolist():
    if not df[col].any():
        del df[col]

In [ ]:
# Replace all non NaN values with 1, all NaN values with 0
df.replace(np.nan, 0, inplace=True)
df.replace(['Y', 'R', 'D', 'A', 'E', 'P', 'M', 'Q', 'U'], 1, inplace=True)
df = df.astype('int64')

df = df.groupby(level=0) \
    .last()

# Count of Total times Voted

In [ ]:
total_votes = df.apply(sum, axis=1).rename('lex_votetotal_c')

# Count of Primary Election Votes

In [ ]:
prim = df[[x for x in df.columns.tolist() if 'primary' in x]]
prim_votes = prim.apply(sum, axis=1).rename('lex_voteprim_c')

# Count of General Election Votes

In [ ]:
gen = df[[x for x in df.columns.tolist() if 'general' in x]]
gen_votes = gen.apply(sum, axis=1).rename('lex_votegen_c')

# Count of Presidential Election Votes

In [ ]:
pres = df[[x for x in df.columns.tolist() if 'pres' in x]]
pres_votes = pres.apply(sum, axis=1).rename('lex_votepres_c')

# Death Vote
Of 9,802 participants, 3,914 voted (40%)

**2791 participants are recorded as voting after deceased**

This corresponds to 28.5% of total participants  
This corresponds to 71.3% of participants that voted

In [ ]:
datecols = ['yrdeath', 'voted_year_1'] 
death = df_full.groupby(level=0).last()[datecols]
death.columns = ['yrdeath', 'most_recent_vote']
death['lex_deathvote'] = death['yrdeath'] > death['most_recent_vote']

In [ ]:
len(death)

In [ ]:
death.lex_deathvote.value_counts()

In [ ]:
# Now check number of total persons that voted
len(death.most_recent_vote.dropna())

# Combine and Write to File

In [ ]:
combined = pd.concat([total_votes, prim_votes, gen_votes, pres_votes, death], axis=1)

In [ ]:
combined.head()

In [ ]:
combined.to_csv(r"C:\Users\jc4673\Documents\Columbia\CHS_Lexis\LexisNexis\derived_vars\derived\voter_derived.csv")